In [1]:

import numpy as np # linear algebra
import pandas as pd # data processing, CSV file I/O (e.g. pd.read_csv)

import os
for dirname, _, filenames in os.walk('dataset'):
    for filename in filenames:
        print(os.path.join(dirname, filename))


dataset\fake\id0_id1_0000.mp4
dataset\fake\id0_id1_0001.mp4
dataset\fake\id0_id1_0002.mp4
dataset\fake\id0_id1_0003.mp4
dataset\fake\id0_id1_0005.mp4
dataset\fake\id0_id1_0006.mp4
dataset\fake\id0_id1_0007.mp4
dataset\fake\id0_id1_0009.mp4
dataset\fake\id0_id2_0000.mp4
dataset\fake\id0_id2_0001.mp4
dataset\fake\id0_id2_0002.mp4
dataset\fake\id0_id2_0003.mp4
dataset\fake\id0_id2_0004.mp4
dataset\fake\id0_id2_0005.mp4
dataset\fake\id0_id2_0006.mp4
dataset\real\id0_0000.mp4
dataset\real\id0_0001.mp4
dataset\real\id0_0002.mp4
dataset\real\id0_0003.mp4
dataset\real\id0_0004.mp4
dataset\real\id0_0005.mp4
dataset\real\id0_0006.mp4
dataset\real\id0_0007.mp4
dataset\real\id0_0008.mp4
dataset\real\id0_0009.mp4
dataset\real\id1_0000.mp4
dataset\real\id1_0001.mp4
dataset\real\id1_0002.mp4
dataset\real\id1_0003.mp4
dataset\real\id1_0004.mp4
dataset\real\id1_0005.mp4
dataset\real\id1_0006.mp4
dataset\real\id1_0007.mp4
dataset\real\id1_0008.mp4
dataset\real\id1_0009.mp4


In [11]:
import os
import numpy as np
import pandas as pd
import tensorflow as tf
from tensorflow.keras import layers, models
import cv2
from sklearn.model_selection import train_test_split
from sklearn.metrics import accuracy_score
import os
import numpy as np
from align.align_trans import (
    get_reference_facial_points,
    warp_and_crop_face,
)
from align.detector import detect_faces
from PIL import Image
from tqdm import tqdm


D:\2024\NOVEMBER\DeepFake\align\align_trans.py:279: SyntaxWarning: "is" with a literal. Did you mean "=="?
  if align_type is 'cv2_affine':
D:\2024\NOVEMBER\DeepFake\align\align_trans.py:282: SyntaxWarning: "is" with a literal. Did you mean "=="?
  elif align_type is 'affine':


In [3]:
import os

# Define correct paths from the output of os.walk
dataset_dir = 'dataset'

real_videos_path = os.path.join(dataset_dir, 'real') 
fake_videos_path = os.path.join(dataset_dir, 'fake')  


real_videos = [os.path.join(real_videos_path, video) for video in os.listdir(real_videos_path)]
fake_videos = [os.path.join(fake_videos_path, video) for video in os.listdir(fake_videos_path)]


video_paths = real_videos + fake_videos
labels = [0] * len(real_videos) + [1] * len(fake_videos)

df = pd.DataFrame({'video_path': video_paths, 'label': labels})

print(f"Found {len(real_videos)} real videos and {len(fake_videos)} fake videos.")



Found 20 real videos and 15 fake videos.


In [9]:
df

,video_path,label
0,dataset\real\id0_0000.mp4,0
1,dataset\real\id0_0001.mp4,0
2,dataset\real\id0_0002.mp4,0
3,dataset\real\id0_0003.mp4,0
4,dataset\real\id0_0004.mp4,0
5,dataset\real\id0_0005.mp4,0
6,dataset\real\id0_0006.mp4,0
7,dataset\real\id0_0007.mp4,0
8,dataset\real\id0_0008.mp4,0
9,dataset\real\id0_0009.mp4,0


In [1]:

train_df, test_df = train_test_split(df, test_size=0.2, random_state=42)
train_df, val_df = train_test_split(train_df, test_size=0.1, random_state=42)

print(f"Training samples: {len(train_df)}, Validation samples: {len(val_df)}, Testing samples: {len(test_df)}")

NameError: name 'train_test_split' is not defined

In [14]:
len(df.values.tolist())

35

In [26]:
def extract_frames(video_path, num_frames=30):
    cap = cv2.VideoCapture(video_path)
    frames = []
    count = 0
    while cap.isOpened() and count < num_frames:
        ret, frame = cap.read()
        if not ret:
            break
        frame = cv2.resize(frame, (224, 224))  # Resize to 224x224
        frames.append(frame)
        count += 1
    
    cap.release()
    return np.array(frames)

def preprocess(imgname,filename):
    
    crop_size = 112
    scale = crop_size / 112.0
    reference = get_reference_facial_points(default_square=True) * scale
    img = Image.open(imgname)
    try: 
                    _, landmarks = detect_faces(img)
                    facial5points = [[landmarks[0][j], landmarks[0][j + 5]] for j in range(5)]
                    warped_face = warp_and_crop_face(
                        np.array(img),
                        facial5points,
                        reference,
                        crop_size=(crop_size, crop_size),
                    )
                    img_warped = Image.fromarray(warped_face)
                    
                    img_warped.save(filename)
    except Exception:
                    print(
                        "{} is discarded due to exception!".format(
                          Exception),
                        )
                    
               
                
count=0

for i in range(len(df.values.tolist())):
    video_frames = extract_frames(df['video_path'].iloc[i])
    labelname="real"
    if(df['label'].iloc[i]==1):
        labelname="fake"
    for frame in video_frames:
        count=count+1
        filepath="preprocess/"+labelname+"/"+str(count)+".jpg"
        cv2.imwrite("1.jpg", frame)
        preprocess("1.jpg",filepath)
        
    
